# PostgreSQL in Python using SQLAlchemy

## The Old Way of Writing Database Code in Python
We're going to use the library `sqlite3` to create a simple database with two tables **Person** and **Address** in the following design:

![db_design](https://www.pythoncentral.io/wp-content/uploads/2013/04/SQLAlchemyPersonAddress.png)

To create tables and insert data into tables, type:

In [1]:
import sqlite3
conn = sqlite3.connect('example.db')
 
c = conn.cursor()
c.execute('''
          CREATE TABLE person
          (id INTEGER PRIMARY KEY ASC, name varchar(250) NOT NULL)
          ''')
c.execute('''
          CREATE TABLE address
          (id INTEGER PRIMARY KEY ASC, street_name varchar(250), street_number varchar(250),
           post_code varchar(250) NOT NULL, person_id INTEGER NOT NULL,
           FOREIGN KEY(person_id) REFERENCES person(id))
          ''')
 
c.execute('''
          INSERT INTO person VALUES(1, 'Sarun Gulyanon')
          ''')
c.execute('''
          INSERT INTO address VALUES(1, 'Newington Road', '15', '12121', 1)
          ''')
 
conn.commit()
conn.close()

To retrieve data from databases, type:

In [2]:
conn = sqlite3.connect('example.db')
c = conn.cursor()
c.execute('SELECT * FROM person')
print(c.fetchall())
c.execute('SELECT * FROM address')
print(c.fetchall())

[(1, 'Sarun Gulyanon')]
[(1, 'Newington Road', '15', '12121', 1)]


In [3]:
c.execute('DROP TABLE person')
c.execute('DROP TABLE address')
conn.close()

----

## Python's SQLAlchemy and Declarative
There are three most important components in writing SQLAlchemy code:

* A Table that represents a table in a database.
* A mapper that maps a Python class to a table in a database.
* A class object that defines how a database record maps to a normal Python object.

Instead of having to write code for Table, mapper and the class object at different places, SQLAlchemy's declarative allows a Table, a mapper and a class object to be defined at once in one class definition.

The following declarative definitions specify the same tables:

In [4]:
!pip install sqlalchemy

In [5]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
 
Base = declarative_base()
 
class Person(Base):
    __tablename__ = 'person'
    # Here we define columns for the table person
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)
    
class Address(Base):
    __tablename__ = 'address'
    # Here we define columns for the table address.
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    street_name = Column(String(250))
    street_number = Column(String(250))
    post_code = Column(String(250), nullable=False)
    person_id = Column(Integer, ForeignKey('person.id'))
    person = relationship(Person)

# Create an engine that stores data in the local directory's
# sqlalchemy_example.db file.
engine = create_engine('sqlite:///sqlalchemy_example.db')
 
# Create all tables in the engine. This is equivalent to "Create Table"
# statements in raw SQL.
Base.metadata.create_all(engine)


Add records into the database.

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
 
engine = create_engine('sqlite:///sqlalchemy_example.db')
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine
 
DBSession = sessionmaker(bind=engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()
 
# Insert a Person in the person table
new_person = Person(name='John Doe')
session.add(new_person)
session.commit()
 
# Insert an Address in the address table
new_address = Address(street_name='North Street', street_number='21', 
                      post_code='46202', person=new_person)
session.add(new_address)
session.commit()

To query, type:

In [7]:
engine = create_engine('sqlite:///sqlalchemy_example.db')
Base.metadata.bind = engine

DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()
# Make a query to find all Persons in the database
print(session.query(Person).all())

# Return the first Person from all Persons in the database
person = session.query(Person).first()
print(person.name)

# Find all Address whose person field is pointing to the person object
print(session.query(Address).filter(Address.person == person).all())

# Retrieve one Address whose person field is point to the person object
print(session.query(Address).filter(Address.person == person).one())

address = session.query(Address).filter(Address.person == person).one()
print(address.id, address.street_name, address.street_number, 
      address.post_code, address.person_id)

John Doe
1 North Street 21 46202 1


----

## SQLAlchemy Core

SQLAlchemy core uses SQL Expression Language that provides a schema-centric usage paradigm

### Connect to Database
Use `create_engine` with this pattern `dialect[+driver]://user:password@host/dbname`:

In [8]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///college.db', echo = True)

### Create Table

Metadata is a collection of Table objects and their associated schema constructs. Use `Table` to define the schema and use `meta.create_all` to create the table:

In [9]:
from sqlalchemy import Table, Column, Integer, String, MetaData
meta = MetaData()

students = Table(
   'students', meta, 
   Column('id', Integer, primary_key = True), 
   Column('name', String), 
   Column('lastname', String), 
)

meta.create_all(engine)

2019-03-25 19:45:25,831 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-03-25 19:45:25,832 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:25,832 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-03-25 19:45:25,833 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:25,834 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("students")
2019-03-25 19:45:25,834 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:25,835 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	lastname VARCHAR, 
	PRIMARY KEY (id)
)


2019-03-25 19:45:25,835 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:25,838 INFO sqlalchemy.engine.base.Engine COMMIT


### Insert/Select Records

1. Construct SQL expressions using the following methods:

In [10]:
ins = students.insert()
print(ins)

INSERT INTO students (id, name, lastname) VALUES (:id, :name, :lastname)


In [11]:
ins = students.insert().values(name = 'Carol', lastname='Danvers')
print(ins)
print(ins.compile().params)

INSERT INTO students (name, lastname) VALUES (:name, :lastname)
{'name': 'Carol', 'lastname': 'Danvers'}


In [12]:
ups = students.update()
print(ups)

UPDATE students SET id=:id, name=:name, lastname=:lastname


In [13]:
des = students.delete()
print(des)

DELETE FROM students


In [14]:
ses = students.select()
print(ses)

SELECT students.id, students.name, students.lastname 
FROM students


2. Execute expressions

Obtain a connection object representing an actively checked out DBAPI connection resource and then execute the expression object.

In [15]:
conn = engine.connect()

ins = students.insert().values(name = 'Bruce', lastname = 'Wayne')
result = conn.execute(ins)

2019-03-25 19:45:25,867 INFO sqlalchemy.engine.base.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2019-03-25 19:45:25,867 INFO sqlalchemy.engine.base.Engine ('Bruce', 'Wayne')
2019-03-25 19:45:25,868 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
conn.execute(students.insert(), [
    {'name':'Peter', 'lastname' : 'Parker'},
    {'name':'Steve','lastname' : 'Rogers'},
    {'name':'Tony','lastname' : 'Stark'},
])

2019-03-25 19:45:25,874 INFO sqlalchemy.engine.base.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2019-03-25 19:45:25,875 INFO sqlalchemy.engine.base.Engine (('Peter', 'Parker'), ('Steve', 'Rogers'), ('Tony', 'Stark'))
2019-03-25 19:45:25,876 INFO sqlalchemy.engine.base.Engine COMMIT


In [17]:
s = students.select()
result = conn.execute(s)
for i, name, lastname in result:
    print('ID: {}, Name: {}, Lastname: {}'.format(i, name, lastname))

2019-03-25 19:45:25,888 INFO sqlalchemy.engine.base.Engine SELECT students.id, students.name, students.lastname 
FROM students
2019-03-25 19:45:25,889 INFO sqlalchemy.engine.base.Engine ()
ID: 1, Name: Bruce, Lastname: Wayne
ID: 2, Name: Peter, Lastname: Parker
ID: 3, Name: Steve, Lastname: Rogers
ID: 4, Name: Tony, Lastname: Stark


In [18]:
s = students.select().where(students.c.id>2)
result = conn.execute(s)
for row in result:
    print(row)

2019-03-25 19:45:25,900 INFO sqlalchemy.engine.base.Engine SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id > ?
2019-03-25 19:45:25,900 INFO sqlalchemy.engine.base.Engine (2,)
(3, 'Steve', 'Rogers')
(4, 'Tony', 'Stark')


### Use SQL

In [19]:
from sqlalchemy import text
t = text("SELECT * FROM students")
result = conn.execute(t)
for row in result:
    print(row)

2019-03-25 19:45:25,908 INFO sqlalchemy.engine.base.Engine SELECT * FROM students
2019-03-25 19:45:25,908 INFO sqlalchemy.engine.base.Engine ()
(1, 'Bruce', 'Wayne')
(2, 'Peter', 'Parker')
(3, 'Steve', 'Rogers')
(4, 'Tony', 'Stark')


In [20]:
s = text("select students.name, students.lastname from students where students.name between :x and :y")
result = conn.execute(s, x = 'A', y = 'Q').fetchall()
for row in result:
    print(row)

2019-03-25 19:45:25,915 INFO sqlalchemy.engine.base.Engine select students.name, students.lastname from students where students.name between ? and ?
2019-03-25 19:45:25,915 INFO sqlalchemy.engine.base.Engine ('A', 'Q')
('Bruce', 'Wayne')
('Peter', 'Parker')


In [21]:
from sqlalchemy import and_
from sqlalchemy.sql import select
s = select([text("* from students")]) \
.where(
   and_(
      text("students.name between :x and :y"),
      text("students.id>1")
   )
)
conn.execute(s, x = 'A', y = 'Q').fetchall()

2019-03-25 19:45:25,922 INFO sqlalchemy.engine.base.Engine SELECT * from students 
WHERE students.name between ? and ? AND students.id>1
2019-03-25 19:45:25,922 INFO sqlalchemy.engine.base.Engine ('A', 'Q')


[(2, 'Peter', 'Parker')]

### Update Records

In [22]:
stmt = students.update() \
        .where(students.c.lastname=='Parker') \
        .values(name='Miles', lastname='Morales')
conn.execute(stmt)
s = students.select()
result = conn.execute(s).fetchall()
for row in result:
    print(row)

2019-03-25 19:45:25,928 INFO sqlalchemy.engine.base.Engine UPDATE students SET name=?, lastname=? WHERE students.lastname = ?
2019-03-25 19:45:25,929 INFO sqlalchemy.engine.base.Engine ('Miles', 'Morales', 'Parker')
2019-03-25 19:45:25,930 INFO sqlalchemy.engine.base.Engine COMMIT
2019-03-25 19:45:25,932 INFO sqlalchemy.engine.base.Engine SELECT students.id, students.name, students.lastname 
FROM students
2019-03-25 19:45:25,933 INFO sqlalchemy.engine.base.Engine ()
(1, 'Bruce', 'Wayne')
(2, 'Miles', 'Morales')
(3, 'Steve', 'Rogers')
(4, 'Tony', 'Stark')


### Delete Records

In [23]:
stmt = students.delete().where(students.c.id > 2)
conn.execute(stmt)
s = students.select()
result = conn.execute(s).fetchall()
for row in result:
    print(row)

2019-03-25 19:45:25,938 INFO sqlalchemy.engine.base.Engine DELETE FROM students WHERE students.id > ?
2019-03-25 19:45:25,939 INFO sqlalchemy.engine.base.Engine (2,)
2019-03-25 19:45:25,940 INFO sqlalchemy.engine.base.Engine COMMIT
2019-03-25 19:45:25,943 INFO sqlalchemy.engine.base.Engine SELECT students.id, students.name, students.lastname 
FROM students
2019-03-25 19:45:25,944 INFO sqlalchemy.engine.base.Engine ()
(1, 'Bruce', 'Wayne')
(2, 'Miles', 'Morales')


### Join
First, lets create another table called `address`:

In [24]:
addresses = Table(
   'addresses', meta, 
   Column('id', Integer, primary_key = True), 
   Column('st_id', Integer), 
   Column('postal_add', String), 
   Column('email_add', String)
)
meta.create_all(engine)

conn.execute(addresses.insert(), [
   {'st_id':1, 'postal_add':'1007 Mountain Drive, Gotham', 'email_add':'batman@avengers.com'},
   {'st_id':3, 'postal_add':'569 Leaman Place', 'email_add':'capamerica@avengers.com'},
   {'st_id':2, 'postal_add':'20 Ingram Street', 'email_add':'spiderman@avengers.com'},
   {'st_id':4, 'postal_add':'10880 Malibu Point', 'email_add':'ironman@avengers.com'},
])

2019-03-25 19:45:25,954 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("students")
2019-03-25 19:45:25,954 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:25,956 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-03-25 19:45:25,956 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:25,957 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	st_id INTEGER, 
	postal_add VARCHAR, 
	email_add VARCHAR, 
	PRIMARY KEY (id)
)


2019-03-25 19:45:25,957 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:25,960 INFO sqlalchemy.engine.base.Engine COMMIT
2019-03-25 19:45:25,961 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (st_id, postal_add, email_add) VALUES (?, ?, ?)
2019-03-25 19:45:25,961 INFO sqlalchemy.engine.base.Engine ((1, '1007 Mountain Drive, Gotham', 'batman@avengers.com'), (3, '569 Leaman Place', 'capamerica@avengers.com'), (2, '20 Ingram Street', 'spiderman@avengers.com'), (4, '10880 Malibu Point', 'i

Either use `select` or `join`

In [25]:
from sqlalchemy.sql import select
s = select([students, addresses]).where(students.c.id == addresses.c.st_id)
result = conn.execute(s)

for row in result:
       print (row)

2019-03-25 19:45:25,970 INFO sqlalchemy.engine.base.Engine SELECT students.id, students.name, students.lastname, addresses.id, addresses.st_id, addresses.postal_add, addresses.email_add 
FROM students, addresses 
WHERE students.id = addresses.st_id
2019-03-25 19:45:25,971 INFO sqlalchemy.engine.base.Engine ()
(1, 'Bruce', 'Wayne', 1, 1, '1007 Mountain Drive, Gotham', 'batman@avengers.com')
(2, 'Miles', 'Morales', 3, 2, '20 Ingram Street', 'spiderman@avengers.com')


In [26]:
from sqlalchemy import join
stmt = students.join(addresses, students.c.id == addresses.c.st_id)
print(stmt)
result = conn.execute(stmt.select())
result.fetchall()

students JOIN addresses ON students.id = addresses.st_id
2019-03-25 19:45:25,978 INFO sqlalchemy.engine.base.Engine SELECT students.id, students.name, students.lastname, addresses.id, addresses.st_id, addresses.postal_add, addresses.email_add 
FROM students JOIN addresses ON students.id = addresses.st_id
2019-03-25 19:45:25,979 INFO sqlalchemy.engine.base.Engine ()


[(1, 'Bruce', 'Wayne', 1, 1, '1007 Mountain Drive, Gotham', 'batman@avengers.com'),
 (2, 'Miles', 'Morales', 3, 2, '20 Ingram Street', 'spiderman@avengers.com')]

### Drop Table

In [27]:
engine.table_names()

2019-03-25 19:45:25,985 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-03-25 19:45:25,986 INFO sqlalchemy.engine.base.Engine ()


['addresses', 'students']

In [28]:
students.drop(engine)
addresses.drop(engine)

2019-03-25 19:45:25,992 INFO sqlalchemy.engine.base.Engine 
DROP TABLE students
2019-03-25 19:45:25,993 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:25,995 INFO sqlalchemy.engine.base.Engine COMMIT
2019-03-25 19:45:25,996 INFO sqlalchemy.engine.base.Engine 
DROP TABLE addresses
2019-03-25 19:45:25,997 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:25,999 INFO sqlalchemy.engine.base.Engine COMMIT


In [29]:
engine.table_names()

2019-03-25 19:45:26,003 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-03-25 19:45:26,004 INFO sqlalchemy.engine.base.Engine ()


[]

----

## SQLAlchemy ORM

SQLAlchemy ORM is a domain-centric mode of usage. ORM API facilitates associating user-defined Python classes with database tables, and objects of those classes with rows in their corresponding tables. Changes in states of objects and rows are synchronously matched with each other. SQLAlchemy enables expressing database queries in terms of user defined classes and their defined relationships.

### Connect Database

In [30]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sales.db', echo = True)

### Declare Mapping

The `declarative_base()` function is used to create base class. A base class stores a catlog of classes and mapped tables in the Declarative system.

In [31]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [32]:
from sqlalchemy import Column, Integer, String

class Customers(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    address = Column(String)
    email = Column(String)

Base.metadata.create_all(engine)

2019-03-25 19:45:26,022 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-03-25 19:45:26,022 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:26,023 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-03-25 19:45:26,023 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:26,024 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("customers")
2019-03-25 19:45:26,024 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:26,025 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE customers (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	address VARCHAR, 
	email VARCHAR, 
	PRIMARY KEY (id)
)


2019-03-25 19:45:26,026 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:26,028 INFO sqlalchemy.engine.base.Engine COMMIT


### Session

A session object is the handle to database, required in order to interact with the database. Session class is defined using `sessionmaker()` – a configurable session factory method which is bound to the engine object created earlier.

In [33]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)
session = Session()

### Insert Records
Insert one record, type:

In [34]:
c1 = Customers(name = 'Bruce Wayne', address = '1007 Mountain Drive, Gotham', email = 'batman@avengers.com')
session.add(c1)
session.commit()

2019-03-25 19:45:26,038 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-25 19:45:26,039 INFO sqlalchemy.engine.base.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2019-03-25 19:45:26,039 INFO sqlalchemy.engine.base.Engine ('Bruce Wayne', '1007 Mountain Drive, Gotham', 'batman@avengers.com')
2019-03-25 19:45:26,041 INFO sqlalchemy.engine.base.Engine COMMIT


For multiple records, type:

In [35]:
session.add_all([
   Customers(name = 'Peter Parker', address = '20 Ingram Street', email = 'spiderman@avengers.com'), 
   Customers(name = 'Steve Rogers', address = '569 Leaman Place', email = 'capamerica@avengers.com'), 
   Customers(name = 'Tony Stark', address = '10880 Malibu Point', email = 'ironman@avengers.com')]
)
session.commit()

2019-03-25 19:45:26,049 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-25 19:45:26,050 INFO sqlalchemy.engine.base.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2019-03-25 19:45:26,050 INFO sqlalchemy.engine.base.Engine ('Peter Parker', '20 Ingram Street', 'spiderman@avengers.com')
2019-03-25 19:45:26,051 INFO sqlalchemy.engine.base.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2019-03-25 19:45:26,051 INFO sqlalchemy.engine.base.Engine ('Steve Rogers', '569 Leaman Place', 'capamerica@avengers.com')
2019-03-25 19:45:26,052 INFO sqlalchemy.engine.base.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2019-03-25 19:45:26,052 INFO sqlalchemy.engine.base.Engine ('Tony Stark', '10880 Malibu Point', 'ironman@avengers.com')
2019-03-25 19:45:26,053 INFO sqlalchemy.engine.base.Engine COMMIT


### Query

In [36]:
result = session.query(Customers).all()
for row in result:
    print("Name: ",row.name, ", Address:",row.address, ", Email:",row.email)

2019-03-25 19:45:26,060 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-25 19:45:26,061 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers
2019-03-25 19:45:26,062 INFO sqlalchemy.engine.base.Engine ()
Name:  Bruce Wayne , Address: 1007 Mountain Drive, Gotham , Email: batman@avengers.com
Name:  Peter Parker , Address: 20 Ingram Street , Email: spiderman@avengers.com
Name:  Steve Rogers , Address: 569 Leaman Place , Email: capamerica@avengers.com
Name:  Tony Stark , Address: 10880 Malibu Point , Email: ironman@avengers.com


In [37]:
from sqlalchemy.orm.query import Query
q = Query(Customers, session)
for row in q:
    print("Name: ",row.name, ", Address:",row.address, ", Email:",row.email)

2019-03-25 19:45:26,068 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers
2019-03-25 19:45:26,069 INFO sqlalchemy.engine.base.Engine ()
Name:  Bruce Wayne , Address: 1007 Mountain Drive, Gotham , Email: batman@avengers.com
Name:  Peter Parker , Address: 20 Ingram Street , Email: spiderman@avengers.com
Name:  Steve Rogers , Address: 569 Leaman Place , Email: capamerica@avengers.com
Name:  Tony Stark , Address: 10880 Malibu Point , Email: ironman@avengers.com


### Filter
Use this pattern `session.query(class).filter(criteria)`.

In [38]:
result = session.query(Customers).filter(Customers.id>2)
for row in result:
    print("Name: ",row.name, ", Address:",row.address, ", Email:",row.email)

2019-03-25 19:45:26,075 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.id > ?
2019-03-25 19:45:26,076 INFO sqlalchemy.engine.base.Engine (2,)
Name:  Steve Rogers , Address: 569 Leaman Place , Email: capamerica@avengers.com
Name:  Tony Stark , Address: 10880 Malibu Point , Email: ironman@avengers.com


### Update Records
Data is not updated until it is committed.

In [39]:
x = session.query(Customers).get(2)
print ("Name: ", x.name, "Address:", x.address, "Email:", x.email)

2019-03-25 19:45:26,083 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.id = ?
2019-03-25 19:45:26,083 INFO sqlalchemy.engine.base.Engine (2,)
Name:  Peter Parker Address: 20 Ingram Street Email: spiderman@avengers.com


In [40]:
x.name = 'Miles Morales'
session.commit()

2019-03-25 19:45:26,088 INFO sqlalchemy.engine.base.Engine UPDATE customers SET name=? WHERE customers.id = ?
2019-03-25 19:45:26,089 INFO sqlalchemy.engine.base.Engine ('Miles Morales', 2)
2019-03-25 19:45:26,090 INFO sqlalchemy.engine.base.Engine COMMIT


In [41]:
x = session.query(Customers).get(2)
print ("Name: ", x.name, "Address:", x.address, "Email:", x.email)

2019-03-25 19:45:26,096 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-25 19:45:26,097 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.id = ?
2019-03-25 19:45:26,097 INFO sqlalchemy.engine.base.Engine (2,)
Name:  Miles Morales Address: 20 Ingram Street Email: spiderman@avengers.com


If you want to discard changes, use `rollback`.

In [42]:
x = session.query(Customers).first()
print ("Name: ", x.name, "Address:", x.address, "Email:", x.email)

2019-03-25 19:45:26,102 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers
 LIMIT ? OFFSET ?
2019-03-25 19:45:26,103 INFO sqlalchemy.engine.base.Engine (1, 0)
Name:  Bruce Wayne Address: 1007 Mountain Drive, Gotham Email: batman@avengers.com


In [43]:
x.name = 'Batman'
print ("Name: ", x.name, "Address:", x.address, "Email:", x.email)

Name:  Batman Address: 1007 Mountain Drive, Gotham Email: batman@avengers.com


In [44]:
session.rollback()
print ("Name: ", x.name, "Address:", x.address, "Email:", x.email)

2019-03-25 19:45:26,111 INFO sqlalchemy.engine.base.Engine ROLLBACK
2019-03-25 19:45:26,112 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-25 19:45:26,113 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.id = ?
2019-03-25 19:45:26,113 INFO sqlalchemy.engine.base.Engine (1,)
Name:  Bruce Wayne Address: 1007 Mountain Drive, Gotham Email: batman@avengers.com


Update multiple records

In [45]:
session.query(Customers).filter(Customers.id != 2). \
update({Customers.name:"Mr."+Customers.name}, synchronize_session = False)

2019-03-25 19:45:26,118 INFO sqlalchemy.engine.base.Engine UPDATE customers SET name=(? || customers.name) WHERE customers.id != ?
2019-03-25 19:45:26,118 INFO sqlalchemy.engine.base.Engine ('Mr.', 2)


3

In [46]:
result = session.query(Customers).all()
for x in result:
    print ("Name: ", x.name, "Address:", x.address, "Email:", x.email)

2019-03-25 19:45:26,123 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers
2019-03-25 19:45:26,124 INFO sqlalchemy.engine.base.Engine ()
Name:  Bruce Wayne Address: 1007 Mountain Drive, Gotham Email: batman@avengers.com
Name:  Miles Morales Address: 20 Ingram Street Email: spiderman@avengers.com
Name:  Mr.Steve Rogers Address: 569 Leaman Place Email: capamerica@avengers.com
Name:  Mr.Tony Stark Address: 10880 Malibu Point Email: ironman@avengers.com


In [47]:
session.close()

2019-03-25 19:45:26,128 INFO sqlalchemy.engine.base.Engine ROLLBACK


### Relationship
A relationship tells the ORM that these classes should be linked together. It uses the foreign key to determine the nature of this linkage. `relationship.back_populates` allows relationship be expressed in reverse. 

In [48]:
from sqlalchemy import create_engine, ForeignKey, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

engine = create_engine('sqlite:///sales.db', echo = True)
Base = declarative_base()

class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    address = Column(String)
    email = Column(String)

    def __repr__(self):
        return "<Customer(name={}, addr={}, email={})>".format(self.name, self.address, self.email)

class Invoice(Base):
    __tablename__ = 'invoices'
    id = Column(Integer, primary_key = True)
    invno = Column(Integer)
    amount = Column(Integer)
    custid = Column(Integer, ForeignKey('customers.id'))
    customer = relationship("Customer", back_populates = "invoices")
    
    def __repr__(self):
        return "<Invoice(invno={}, amount={}, custid={})>".format(self.invno, self.amount, self.custid)

Customer.invoices = relationship("Invoice", order_by=Invoice.id, back_populates="customer")
Base.metadata.create_all(engine)

2019-03-25 19:45:26,141 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-03-25 19:45:26,142 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:26,142 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-03-25 19:45:26,143 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:26,143 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("customers")
2019-03-25 19:45:26,143 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:26,144 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("invoices")
2019-03-25 19:45:26,145 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:26,145 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE invoices (
	id INTEGER NOT NULL, 
	invno INTEGER, 
	amount INTEGER, 
	custid INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(custid) REFERENCES customers (id)
)


2019-03-25 19:45:26,146 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:26,148 INFO sqlalchemy.en

In [49]:
mary = Customer(name='Mary', address='21 North Street', email='mary@gmail.com')
mary.invoices

[]

In [50]:
mary.invoices = [Invoice(invno=123, amount=500),
                 Invoice(invno=273, amount=1000)]

You can query both ends of the relationship.

In [51]:
mary.invoices[1]

<Invoice(invno=273, amount=1000, custid=None)>

In [52]:
mary.invoices[1].customer

<Customer(name=Mary, addr=21 North Street, email=mary@gmail.com)>

In [53]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)
session = Session()
session.add(mary)
session.commit()

2019-03-25 19:45:26,172 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-25 19:45:26,173 INFO sqlalchemy.engine.base.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2019-03-25 19:45:26,174 INFO sqlalchemy.engine.base.Engine ('Mary', '21 North Street', 'mary@gmail.com')
2019-03-25 19:45:26,175 INFO sqlalchemy.engine.base.Engine INSERT INTO invoices (invno, amount, custid) VALUES (?, ?, ?)
2019-03-25 19:45:26,175 INFO sqlalchemy.engine.base.Engine (123, 500, 5)
2019-03-25 19:45:26,176 INFO sqlalchemy.engine.base.Engine INSERT INTO invoices (invno, amount, custid) VALUES (?, ?, ?)
2019-03-25 19:45:26,176 INFO sqlalchemy.engine.base.Engine (273, 1000, 5)
2019-03-25 19:45:26,177 INFO sqlalchemy.engine.base.Engine COMMIT


In [54]:
mary = session.query(Customer).filter_by(name='Mary').one()
mary

2019-03-25 19:45:26,183 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-25 19:45:26,183 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.name = ?
2019-03-25 19:45:26,184 INFO sqlalchemy.engine.base.Engine ('Mary',)


<Customer(name=Mary, addr=21 North Street, email=mary@gmail.com)>

ORM resolves the foreign key for you. When we accessed the invoice collection, SQL was suddenly issued. This is an example of a **lazy loading** relationship.

In [55]:
mary.invoices

2019-03-25 19:45:26,190 INFO sqlalchemy.engine.base.Engine SELECT invoices.id AS invoices_id, invoices.invno AS invoices_invno, invoices.amount AS invoices_amount, invoices.custid AS invoices_custid 
FROM invoices 
WHERE ? = invoices.custid ORDER BY invoices.id
2019-03-25 19:45:26,191 INFO sqlalchemy.engine.base.Engine (5,)


[<Invoice(invno=123, amount=500, custid=5)>,
 <Invoice(invno=273, amount=1000, custid=5)>]

In [56]:
for u, a in session.query(Customer, Invoice).\
        filter(Customer.id==Invoice.custid).\
        all():
    print(u, a)

2019-03-25 19:45:26,197 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email, invoices.id AS invoices_id, invoices.invno AS invoices_invno, invoices.amount AS invoices_amount, invoices.custid AS invoices_custid 
FROM customers, invoices 
WHERE customers.id = invoices.custid
2019-03-25 19:45:26,197 INFO sqlalchemy.engine.base.Engine ()
<Customer(name=Mary, addr=21 North Street, email=mary@gmail.com)> <Invoice(invno=123, amount=500, custid=5)>
<Customer(name=Mary, addr=21 North Street, email=mary@gmail.com)> <Invoice(invno=273, amount=1000, custid=5)>


#### Eager Loading
To reduce the number of queries, apply an **eager load** to the query operation (at the cost of "heavier" call).

In [57]:
from sqlalchemy.orm import joinedload

mary = session.query(Customer).\
        options(joinedload(Customer.invoices)).\
        filter(Customer.name == 'Mary').\
        one()
mary

2019-03-25 19:45:26,204 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email, invoices_1.id AS invoices_1_id, invoices_1.invno AS invoices_1_invno, invoices_1.amount AS invoices_1_amount, invoices_1.custid AS invoices_1_custid 
FROM customers LEFT OUTER JOIN invoices AS invoices_1 ON customers.id = invoices_1.custid 
WHERE customers.name = ? ORDER BY invoices_1.id
2019-03-25 19:45:26,204 INFO sqlalchemy.engine.base.Engine ('Mary',)


<Customer(name=Mary, addr=21 North Street, email=mary@gmail.com)>

In [58]:
mary.invoices

[<Invoice(invno=123, amount=500, custid=5)>,
 <Invoice(invno=273, amount=1000, custid=5)>]

### Delete Records

In [59]:
session.delete(mary)
session.query(Customer).filter_by(name='Mary').count()

2019-03-25 19:45:26,216 INFO sqlalchemy.engine.base.Engine UPDATE invoices SET custid=? WHERE invoices.id = ?
2019-03-25 19:45:26,217 INFO sqlalchemy.engine.base.Engine ((None, 1), (None, 2))
2019-03-25 19:45:26,218 INFO sqlalchemy.engine.base.Engine DELETE FROM customers WHERE customers.id = ?
2019-03-25 19:45:26,218 INFO sqlalchemy.engine.base.Engine (5,)
2019-03-25 19:45:26,219 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.name = ?) AS anon_1
2019-03-25 19:45:26,220 INFO sqlalchemy.engine.base.Engine ('Mary',)


0

In [60]:
session.query(Invoice).count()

2019-03-25 19:45:26,226 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT invoices.id AS invoices_id, invoices.invno AS invoices_invno, invoices.amount AS invoices_amount, invoices.custid AS invoices_custid 
FROM invoices) AS anon_1
2019-03-25 19:45:26,228 INFO sqlalchemy.engine.base.Engine ()


2

In [61]:
session.close()

2019-03-25 19:45:26,232 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [62]:
Base = declarative_base()

class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    address = Column(String)
    email = Column(String)
    invoices = relationship("Invoice", back_populates='customer',\
                            cascade="all, delete, delete-orphan")

    def __repr__(self):
        return "<Customer(name={}, addr={}, email={})>".format(self.name, self.address, self.email)

class Invoice(Base):
    __tablename__ = 'invoices'
    id = Column(Integer, primary_key = True)
    invno = Column(Integer)
    amount = Column(Integer)
    custid = Column(Integer, ForeignKey('customers.id'))
    customer = relationship("Customer", back_populates = "invoices")
    
    def __repr__(self):
        return "<Invoice(invno={}, amount={}, custid={})>".format(self.invno, self.amount, self.custid)

In [63]:
mary = session.query(Customer).filter_by(name='Mary').one()
mary

2019-03-25 19:45:26,247 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-25 19:45:26,248 INFO sqlalchemy.engine.base.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.name = ?
2019-03-25 19:45:26,249 INFO sqlalchemy.engine.base.Engine ('Mary',)


<Customer(name=Mary, addr=21 North Street, email=mary@gmail.com)>

In [64]:
session.delete(mary)

2019-03-25 19:45:26,254 INFO sqlalchemy.engine.base.Engine SELECT invoices.id AS invoices_id, invoices.invno AS invoices_invno, invoices.amount AS invoices_amount, invoices.custid AS invoices_custid 
FROM invoices 
WHERE ? = invoices.custid
2019-03-25 19:45:26,255 INFO sqlalchemy.engine.base.Engine (5,)


In [65]:
print(session.query(Customer).filter_by(name='Mary').count())
print(session.query(Invoice).count())

2019-03-25 19:45:26,262 INFO sqlalchemy.engine.base.Engine DELETE FROM invoices WHERE invoices.id = ?
2019-03-25 19:45:26,262 INFO sqlalchemy.engine.base.Engine ((1,), (2,))
2019-03-25 19:45:26,264 INFO sqlalchemy.engine.base.Engine DELETE FROM customers WHERE customers.id = ?
2019-03-25 19:45:26,264 INFO sqlalchemy.engine.base.Engine (5,)
2019-03-25 19:45:26,266 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.name = ?) AS anon_1
2019-03-25 19:45:26,267 INFO sqlalchemy.engine.base.Engine ('Mary',)
0
2019-03-25 19:45:26,269 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT invoices.id AS invoices_id, invoices.invno AS invoices_invno, invoices.amount AS invoices_amount, invoices.custid AS invoices_custid 
FROM invoices) AS anon_1
2019-03-25 19:45:26,269 INFO sqlalch

In [66]:
session.close()

2019-03-25 19:45:26,274 INFO sqlalchemy.engine.base.Engine ROLLBACK


### Drop Table

In [67]:
engine.table_names()

2019-03-25 19:45:26,279 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-03-25 19:45:26,280 INFO sqlalchemy.engine.base.Engine ()


['customers', 'invoices']

In [68]:
Customer.__table__.drop(engine)
Invoice.__table__.drop(engine)

2019-03-25 19:45:26,288 INFO sqlalchemy.engine.base.Engine 
DROP TABLE customers
2019-03-25 19:45:26,289 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:26,292 INFO sqlalchemy.engine.base.Engine COMMIT
2019-03-25 19:45:26,292 INFO sqlalchemy.engine.base.Engine 
DROP TABLE invoices
2019-03-25 19:45:26,293 INFO sqlalchemy.engine.base.Engine ()
2019-03-25 19:45:26,295 INFO sqlalchemy.engine.base.Engine COMMIT


In [69]:
engine.table_names()

2019-03-25 19:45:26,300 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-03-25 19:45:26,301 INFO sqlalchemy.engine.base.Engine ()


[]